<a href="https://colab.research.google.com/github/shiyu-cheng/6895_Visual_Sentiment_Prediction/blob/master/image_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install flickrapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 5.3 MB/s eta 0:00:00


In [4]:
import csv
from pprint import pprint
from flickrapi import FlickrAPI
import urllib
from PIL import Image
import os
from time import sleep

In [5]:
directory = '/content/drive/MyDrive/6895/'

In [22]:
feelings = (
    "Joy",
    "Trust",
    "Fear",
    "Surprise",
    "Sadness",
    "Disgust",
    "Anger",
    "Anticipation",
    "Love",
    "Submission",
    "Awe",
    "Disapproval",
    "Remorse",
    "Contempt",
    "Aggressiveness",
    "Optimism",
    "Guilt",
    "Shame",
    "Envy",
    "Indifference",
    "Serenity",
    "Boredom",
    "Irritation",
    "Pessimism"
)

In [36]:
#search images
FLICKR_PUBLIC = 'f8b9cd454275fca40682a0e602c67073'
FLICKR_SECRET = 'cab676a88194ebae'

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
#ATTENTION!!!
#the flickr api has hugs! the offsets of image size categories are wrong... 
#To get url_s, which is the URL of small suqare 75x75 size image, 
#we have to use parameter url_sq
extras = 'url_sq' #URL of small suqare 75x75 size image
per_page = 100
page = range(1,6)
img_sz = 75
feeling_dict = {}
for feeling in feelings:
  url_list = []
  for i in page:
    cats = flickr.photos.search(tags = feeling, per_page = per_page, page = i, extras = extras, sort = 'relevance')
    photos = cats['photos']
    # print(type(photos['photo']))
    # pprint(photos)
    for p in photos['photo']: #dict
      url = p['url_sq']
      height = p['height_sq']
      width = p['width_sq']
      if height != img_sz or width != img_sz:
        continue
      
      url_list.append(url)

  feeling_dict[feeling] = url_list
# print(url_list)

In [37]:
#download image
# feeling_dr_set = set()
img_file = 'images_flickr_tag_sort_by_relevance/'
isExist = os.path.exists(directory + img_file)
if not isExist:
  os.mkdir(directory + img_file)
for f in feeling_dict:
  img_dr = directory + img_file + f
  isExist = os.path.exists(img_dr)
  if not isExist:
    os.mkdir(img_dr)
  # feeling_dr_set.add(img_dr)
  for url in feeling_dict[f]:
    img_name = url.split('/')[-1]
    path = img_dr + '/' + img_name
    if os.path.exists(path):
      continue
    # sleep(1)
    try:
      urllib.request.urlretrieve(url, path)
      urllib.request.urlcleanup()
    except Exception as e: 
      print(e)

# Resize the image and overwrite it
# image = Image.open('00001.jpg') 
# image = image.resize((img_sz, img_sz), Image.ANTIALIAS)
# image.save('00001.jpg')

HTTP Error 504: Gateway Time-out
HTTP Error 504: Gateway Time-out


In [ ]:
#image to csv
import numpy as np
import matplotlib.image as img
import os
import pandas as pd

csv_file = os.path.join(directory, 'images_flickr_tag/flickr_tag_relevence_img.csv')
if os.path.exists(csv_file):
  os.remove(os.path.join(directory, 'images_flickr_tag/flickr_tag_relevence_img.csv'))

# iterate over files
img_dr = directory + '/images_flickr_tag_sort_by_relevance/'
for file in os.listdir(img_dr):
  file = os.path.join(img_dr, file)
  for filename in os.listdir(file):
    # read an image
    imageMat = img.imread(os.path.join(file, filename))
    print("Image shape:", imageMat.shape)
    
    # if image is colored (RGB)
    if(imageMat.ndim == 3 and imageMat.shape[2] == 3):
      
      # reshape it from 3D matrice to 1D arraye
      imageMat_reshape = imageMat.reshape(1,
                                          -1)
      print("Reshaping to 2D array:",
            imageMat_reshape.shape)
    
    # if image is grayscale
    else:
      continue
        
    mat_df = pd.DataFrame(imageMat_reshape)
    mat_df['label'] = file.split('/')[-1]
 
    # exporting dataframe to CSV file.
    # mat_df.to_csv(os.path.join(directory, 'images/test2_img.csv'),
    #               header = None,
    #               index = None)
    
    if not os.path.exists(csv_file):
      mat_df.to_csv(csv_file, index=False)
    else:
      mat_df.to_csv(csv_file, mode='a', index=False, header=False)
    


Next step: use unsplash to update the dataset
Awe

In [24]:
# # retrieving matrice from the .csv file
# loaded_2D_mat = np.loadtxt(directory + 'test_img.csv')
 
# # reshaping it to 3D matrice
# loaded_mat = loaded_2D_mat.reshape(loaded_2D_mat.shape[0],
#                                    loaded_2D_mat.shape[1] // imageMat.shape[2],
#                                    imageMat.shape[2])
 
# print("Image shape of loaded Image:",
#       loaded_mat.shape)
 
# # check if both matrice have same shape or not
# if((imageMat == loaded_mat).all()):
   
#   print("\n\nYes",
#         "The loaded matrice from CSV file is same as original image matrice")

Image shape of loaded Image: (75, 75, 3)


Yes The loaded matrice from CSV file is same as original image matrice
